In [24]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.io


In [2]:
input_2 = pd.read_csv('input_2.csv')

input_2 = input_2.values

In [3]:
input_2_res = input_2.reshape(6000, 3, 6001)

input_2_res.shape

(6000, 3, 6001)

In [4]:
a, b, c = input_2_res.shape  # 6000, 3, 6001

# Crear array vacío para los datos normalizados con misma forma
datos_normalizados = np.zeros_like(input_2_res)

scaler = MinMaxScaler()

for i in range(b): 
    # Tomamos todas las muestras para el canal i, forma (6000, 6001)
    datos_canal = input_2_res[:, i, :]
    datos_canal_norm = scaler.fit_transform(datos_canal)

    # Guardamos la normalización en el resultado
    datos_normalizados[:, i, :] = datos_canal_norm

In [5]:
datos_normalizados

array([[[0.00000000e+00, 4.67751556e-01, 5.73972182e-01, ...,
         6.00396138e-01, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 5.16507807e-01, 5.06759789e-01, ...,
         5.58031939e-01, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 5.61223056e-01, 4.86514237e-01, ...,
         4.54977361e-01, 0.00000000e+00, 0.00000000e+00]],

       [[1.66694449e-04, 4.67747122e-01, 5.74102094e-01, ...,
         6.00398208e-01, 0.00000000e+00, 0.00000000e+00],
        [1.66694449e-04, 5.16502970e-01, 5.06907610e-01, ...,
         5.58038347e-01, 0.00000000e+00, 0.00000000e+00],
        [1.66694449e-04, 5.61216665e-01, 4.86725553e-01, ...,
         4.54984023e-01, 0.00000000e+00, 0.00000000e+00]],

       [[3.33388898e-04, 4.67741950e-01, 5.74298383e-01, ...,
         6.00403788e-01, 0.00000000e+00, 0.00000000e+00],
        [3.33388898e-04, 5.16495506e-01, 5.07112949e-01, ...,
         5.58045207e-01, 0.00000000e+00, 0.00000000e+00],
        [3.33388898e-04, 5.61217354e

In [23]:
datos_normalizados.shape

(6000, 3, 6001)

In [6]:
output = pd.read_csv('output_2.csv')

output

,Unnamed: 0,p_arrival_sample,s_arrival_sample
0,0,400.0,7.119410e+02
1,1,601.0,1.660000e+03
2,2,700.0,1.204556e+03
3,3,400.0,1.286595e+03
4,4,600.0,1.307000e+03
...,...,...,...
5995,5995,0.0,0.000000e+00
5996,5996,0.0,0.000000e+00
5997,5997,900.0,1.109000e+03
5998,5998,1243.8,0.000000e+00


In [25]:
class SismoDataset(Dataset):
    def __init__(self, X, y):
        # X: tensor (samples, channels, length)
        # y: tensor (samples, 2)
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Conversión a tensores torch
X_tensor = torch.tensor(datos_normalizados, dtype=torch.float32)  # (6000, 3, 6001)
y_tensor = torch.tensor(targets, dtype=torch.float32)             # (6000, 2)

dataset = SismoDataset(X_tensor, y_tensor)

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SpectralConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1):
        super(SpectralConv1d, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1

        self.scale = (1 / (in_channels*out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, dtype=torch.cfloat))

    def compl_mul1d(self, input, weights):
        # Multiplicación compleja espectral
        return torch.einsum("bix,iox->box", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        x_ft = torch.fft.rfft(x)

        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-1)//2 + 1, device=x.device, dtype=torch.cfloat)
        out_ft[:, :, :self.modes1] = self.compl_mul1d(x_ft[:, :, :self.modes1], self.weights1)
        
        x = torch.fft.irfft(out_ft, n=x.size(-1))
        return x

class FNO1dRegression(nn.Module):
    def __init__(self, modes, width, output_dim=2):
        super(FNO1dRegression, self).__init__()

        self.modes1 = modes
        self.width = width
        self.output_dim = output_dim

        self.fc0 = nn.Linear(3, self.width)  # 3 canales/features de entrada

        self.convs = nn.ModuleList([SpectralConv1d(self.width, self.width, self.modes1) for _ in range(4)])
        self.ws = nn.ModuleList([nn.Conv1d(self.width, self.width, 1) for _ in range(4)])

        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, self.output_dim)

    def forward(self, x):
        # x: (batch, channels=3, length=6001)
        # Pero fc0 espera último dim features, lo permutamos:
        x = x.permute(0, 2, 1)  # ahora (batch, length, channels=3)
        x = self.fc0(x)         # (batch, length, width)

        x = x.permute(0, 2, 1)  # (batch, width, length)

        for conv, w in zip(self.convs, self.ws):
            x1 = conv(x)
            x2 = w(x)
            x = x1 + x2
            x = F.relu(x)

        # Pooling global en dimensión length para obtener vector (batch, width)
        x = x.mean(dim=2)  # promedio sobre length

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)  # (batch, output_dim=2)

        return x


In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = FNO1dRegression(modes=16, width=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()


In [30]:
def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0.0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x.size(0)
    return total_loss / len(loader.dataset)

def test_epoch(model, loader, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = loss_fn(pred, y)
            total_loss += loss.item() * x.size(0)
    return total_loss / len(loader.dataset)


In [31]:
for epoch in range(20):
    train_loss = train_epoch(model, train_loader, optimizer, device)
    test_loss = test_epoch(model, test_loader, device)
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Test Loss = {test_loss:.4f}")


Epoch 1: Train Loss = 524651.4759, Test Loss = 330748.0446
Epoch 2: Train Loss = 329026.0189, Test Loss = 335798.6400
